In [1]:
import subprocess
import numpy as np

In [2]:
# Variable params
intro_time = 12 * 5 # intro time for additive
sim_duration = 12 * 100 # might need longer for transitions
trans_time = 12 * 30 # transition time for EG01 --> EGXX
# Transition time is assumed to always be greater than intro time

# "Non-Variable" Params
n_init_rxtrs = 100
n_assem_core = 3
assem_size = 29565
init_total_assem_size = n_init_rxtrs * n_assem_core * assem_size
cycle_time = 18
rxtr_life = 12 * 40 

unblended_UF6 = 2e8

In [3]:
def run_sim(filebase, sim):
    in_file = filebase + '.py'
    sim_file = '../output/' + filebase + '.sqlite'

    with open(in_file, 'w') as file: 
        file.write('SIMULATION = ' + str(sim))
    subprocess.run(['rm', sim_file])
    subprocess.run(['cyclus', in_file, '-o', sim_file, '-v1'])    
    return

# EG01 

## Protoype Definitions

In [4]:
init_uox_source = {'name' : 'SourceInitUOX', 
                   'config' : {'Source' : {'outcommod' : 'c_InitUOX',
                                           'outrecipe' : 'r_UOX',
                                           'inventory_size' : init_total_assem_size
                                          }
                              }
                  }

In [5]:
from eg01_facilities import non_source

In [6]:
# play around with contraining additive source throughput
#add_source['config']['Source']['throughput'] = 1e5

In [7]:
# LWR prototype for full additive availability upon introduction
lwr_full = {'name' : 'LWR',
            'lifetime' : rxtr_life,
            'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['c_UOX_ADD', 'c_UOX', 'c_InitUOX']},
                                     'fuel_outcommods' : {'val' : ['c_S_UOX_ADD', 'c_S_UOX', 'c_S_UOX']},
                                     'fuel_inrecipes' :  {'val' : ['r_UOX_ADD', 'r_UOX', 'r_UOX']},
                                     'fuel_outrecipes' : {'val' : ['r_S_UOX_ADD', 'r_S_UOX', 'r_S_UOX']},
                                     'fuel_prefs' : {'val' : [10, 20, 25]},
#                                      'pref_change_times' : {'val' : intro_time},
#                                      'pref_change_commods' : {'val' : 'c_UOX_ADD'},
#                                      'pref_change_values' : {'val' : 10000},
                                     'cycle_time' : cycle_time,
                                     'refuel_time' : 0,
                                     'assem_size' : assem_size,
                                     'n_assem_core' : n_assem_core,
                                     'n_assem_batch' : 1,
                                     'power_name' : 'PowerLWR',
                                     'power_cap' : 1000
                                    }
                       }
           }

## Regions and Institutions

### 1. Init LWR Fleet (Deploy Inst)

In [8]:
init_lwr_prototypes = ['LWR' for x in range(0, n_init_rxtrs)]
n_builds = [1 for x in range(0, n_init_rxtrs)]
# staggering build times over first 18 timesteps so that reactors 
# don't all cycle together
build_times = [x + 1 for x in range(0, 17) for y in range(0,6)]
del build_times[-3:-1]
# Lifetimes borrowed from previous EG scenario work presuming a 
# start year of 2000 and first decommission in 2015. Updated to 
# better stagger decommissioning.
old_lives = [181, 186, 191, 196, 
             201, 206, 211, 216, 221, 226, 231, 236, 241, 246, 251, 256, 261, 266, 271, 276, 281, 286, 291, 296,
             301, 306, 311, 316, 321, 326, 331, 336, 341, 346, 351, 356, 361, 366, 371, 376, 381, 386, 391, 396,
             401, 406, 411, 416, 421, 426, 431, 436, 441, 446, 451, 456, 461, 466, 471, 476, 481, 486, 491, 496,
             501, 506, 511, 516, 521, 526, 531, 536, 541, 546, 551, 556, 561, 566, 571, 576, 581, 586, 591, 596,
             601, 606, 611, 616, 621, 626, 631, 636, 641, 646, 651, 656, 661, 666, 671, 676
            ]
# Overwrite lifetimes to have decommission start 1.5 year in, 
# expecting a sim start of 2022
lifetimes = [x - 163 for x in old_lives]
lifetimes_y = [x/12. for x in old_lives]
assert len(init_lwr_prototypes) == n_init_rxtrs
assert len(n_builds) == n_init_rxtrs
assert len(build_times) == n_init_rxtrs
assert len(lifetimes) == n_init_rxtrs

print (len(init_lwr_prototypes))

100


In [9]:
init_fleet = {'name' : 'InitFleet', 
              'config' : {'DeployInst' : {'prototypes' :  {'val' : init_lwr_prototypes},
                                          'n_build' :     {'val' : n_builds},
                                          'build_times' : {'val' : build_times},
                                          'lifetimes' :   {'val' : lifetimes}
                                         }
                         }
             }

### 2. EG01 FC Facilities: Manager Inst

In [10]:
eg1_fc_prototypes = ['Source_UNAT', 'SourceNonIsos', 'Enrichment', 
                        'Source_UNAT_ADD', 'Enrichment_ADD', 
                        'StorageDepU', 
                        'LWR', 'UOXCool', 'UOXCool_ADD', 'Waste'
                    ]

eg1_fc_inst = {'name' : 'FCInstEG01', 
               'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'SourceInitUOX'},
                                                      {'number' : 1, 'prototype' : 'Source_UNAT'},
                                                      {'number' : 1, 'prototype' : 'Source_UNAT_ADD'},
                                                      {'number' : 1, 'prototype' : 'SourceNonIsos'},
                                                      {'number' : 1, 'prototype' : 'Enrichment'},
                                                      {'number' : 1, 'prototype' : 'Enrichment_ADD'},
                                                      {'number' : 1, 'prototype' : 'StorageDepU'},
                                                      {'number' : 1, 'prototype' : 'UOXCool'},
                                                      {'number' : 1, 'prototype' : 'UOXCool_ADD'},
                                                      {'number' : 1, 'prototype' : 'Waste'}]
                                       },
               'config' : {'ManagerInst' : {'prototypes' : {'val' : eg1_fc_prototypes}}}
              }

### 3. Growth Regions: Pick Flat or 1% Growth

In [11]:
# 1% growth per year

month_grow_rate = 0.01 / 12
exp_str = '100000 ' + str(month_grow_rate) + ' 0'
exp_func = {'piece' : [{'start' : 18,
                        'function' : {'type' : 'exponential', 'params' : exp_str}
                       }
                      ]
           }
growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'PowerLWR',
                                                          'piecewise_function' : exp_func
                                                         }
                                                        ]
                                              }
                                  }
                }

In [12]:
# flat
# lin_func = {'piece' : [{'start' : 18,
#                         'function' : {'type' : 'linear', 'params' : '0 100000'}
#                        }
#                       ]
#            }
# growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'PowerLWR',
#                                                           'piecewise_function' : lin_func
#                                                          }
#                                                         ]
#                                               }
#                                   }
#                 }

# Recipes
-  Here: 
  1. Depleted U
  2. Natural U
-  100 ppt Init U232 Additive Recipes (in recipe_100ppt.py):
  1. NonAdditive U Isotopes (U234)
  2. Additive U Isotopes (U232, U233, U234)
  3. Almost UOX NonAdditive Enr Ratio
  4. Almost UOX Additive Enr Ratio
  5. UOX without Additive
  6. UOX with Additive
  7. Spent UOX from #5
  8. Spent UOX from #6

In [13]:
mass ={
    'U232' : 234.0409503,
    'U233' : 234.0409503,
    'U234' : 234.0409503,
    'U235' : 234.0409503,
    'U236' : 234.0409503,
    'U238' : 234.0409503
}


# UNAT
UNAT_COMP = {
    'U234':0.000053,
    'U235':0.007110,
    'U238':0.992890
    }


nat_u = {'name' : 'r_UNAT',
         'basis' : 'mass',
         'nuclide' : [
             {'id' : 'U234', 'comp' : UNAT_COMP['U234']}, 
             {'id' : 'U235', 'comp' : UNAT_COMP['U235']}, 
             {'id' : 'U238', 'comp' : UNAT_COMP['U238']}]
        }

source_UNAT = {'name' : 'Source_UNAT',
               'config' : {'Source' : {'outcommod' : 'c_UNAT',
                                       'outrecipe' : 'r_UNAT',
                                       'throughput' : 1.8e11,
                                       'inventory_size' : unblended_UF6
                                      }
                          }
              }

#LWR recipes

UOX_COMP = {
    'U234' : 0.0004631049477,
    'U235' : 0.0499062302930,
    'U238' : 0.9496306647594}

r_UOX = {'name' : 'r_UOX',
             'basis' : 'mass',
             'nuclide' : [
                 {'id' : 'U234', 'comp' : UOX_COMP['U234']},
                 {'id' : 'U235', 'comp' : UOX_COMP['U235']}, 
                 {'id' : 'U238', 'comp' : UOX_COMP['U238']}]
            }


# All non-U values are fake, and U238 is reduced from 
# 0.9760572827708 to accomodate these fake values
r_S_UOX = {'name' : 'r_S_UOX',
               'basis' : 'mass',
               'nuclide' : [{'id' : 'U232',  'comp' : 0.0000000002838},
                            {'id' : 'U233',  'comp' : 0.0000000058245},
                            {'id' : 'U234',  'comp' : 0.0002961396326}, 
                            {'id' : 'U235',  'comp' : 0.0174611824759},
                            {'id' : 'U236',  'comp' : 0.0061853889980},
                            {'id' : 'U238',  'comp' : 0.9250572827708},
                            {'id' : 'Pu239', 'comp' : 0.0100000000000},
                            {'id' : 'Am241', 'comp' : 0.0010000000000},
                            {'id' : 'Cs137', 'comp' : 0.0400000000000}]
              }



# ENRICHMENT NO-ADDITIVE
feed_needed = 10.304 # From URENCO to prod 1kg of UOX at 5% with Tail at 0.25%
effs = []
for nuc in UNAT_COMP:
    eff = (UOX_COMP[nuc])/(UNAT_COMP[nuc]*feed_needed)
    effs.append({'comp': nuc, 'eff': eff})
 
Enrich_UNAT = {'name': 'Enrichment',
           'config': {'Separations': {'leftover_commod': 'c_DU',
                                      'throughput': 5e6,
                                      'feedbuf_size': 1e9,
                                      # can't use feed recipe unless we split
                                      # into 2 sep facilities for each commod
                                      'feed_commods':
                                          {'val': ['c_UNAT']},
                                      'streams': {
                                          'item': [{
                                              'commod': 'c_UOX', 'info': {
                                                  'buf_size': 2.5e5, 'efficiencies':
                                                  {'item': effs}}}]}
                                      }
                      }
           }








# U NAT + ADDITIVE
UNAT_ADD_COMP = {
    'U232':0.0000000001,
    'U233':0.000000000124824092316352,
    'U234':0.0000530944854491094,
    'U235':0.0071095114738325,
    'U238':0.992837393814684
    } # Compo at 100ppt of additive

nat_u_additive = {'name' : 'r_UNAT_ADD',
              'basis' : 'mass',
              'nuclide' : [
                 {'id' : 'U232', 'comp' : UNAT_ADD_COMP['U232']}, 
                 {'id' : 'U233', 'comp' : UNAT_ADD_COMP['U233']}, 
                 {'id' : 'U234', 'comp' : UNAT_ADD_COMP['U234']}, 
                 {'id' : 'U235', 'comp' : UNAT_ADD_COMP['U235']}, 
                 {'id' : 'U238', 'comp' : UNAT_ADD_COMP['U238']}]
        }
source_UNAT_ADD = {'name' : 'Source_UNAT_ADD',
                   'config' : {'Source' : {'outcommod' : 'c_UNAT_ADD',
                                           'outrecipe' : 'r_UNAT_ADD',
                                           'throughput' : 2.1e11
                                          }
                              }
                  }




UOX_ADD_COMP = {
    'U232' : 0.000000000979787642142,
    'U233' : 0.00000000117052802555,
    'U234' : 0.000453772218252,
    'U235' : 0.0499987054095,
    'U238' : 0.949547520216}


r_UOX_ADD = {'name' : 'r_UOX_ADD',
           'basis' : 'mass',
           'nuclide' : [{'id' : 'U232', 'comp' : UOX_ADD_COMP['U232']},
                        {'id' : 'U233', 'comp' : UOX_ADD_COMP['U233']},
                        {'id' : 'U234', 'comp' : UOX_ADD_COMP['U234']}, 
                        {'id' : 'U235', 'comp' : UOX_ADD_COMP['U235']},
                        {'id' : 'U238', 'comp' : UOX_ADD_COMP['U238']}]
          }


# All non-U values are fake, and U238 is reduced from 
# 0.9760334572009 to accomodate these fake values
r_S_UOX_ADD = {'name' : 'r_S_UOX_ADD',
             'basis' : 'mass',
             'nuclide' : [{'id' : 'U232',  'comp' : 0.0000000009065},
                          {'id' : 'U233',  'comp' : 0.0000000062943},
                          {'id' : 'U234',  'comp' : 0.0002327879338}, 
                          {'id' : 'U235',  'comp' : 0.0175266370423},
                          {'id' : 'U236',  'comp' : 0.0062071106222},
                          {'id' : 'U238',  'comp' : 0.9250334572009},
                          {'id' : 'Pu239', 'comp' : 0.0100000000000},
                          {'id' : 'Am241', 'comp' : 0.0010000000000},
                          {'id' : 'Cs137', 'comp' : 0.0400000000000}]
            }




# ENRICHMENT NO-ADDITIVE
feed_needed = 10.304 # From URENCO to prod 1kg of UOX at 5% with Tail at 0.25%
effs_ADD = []
for nuc in UNAT_ADD_COMP:
    eff = (UOX_ADD_COMP[nuc])/(UNAT_ADD_COMP[nuc]*feed_needed)
    effs_ADD.append({'comp': nuc, 'eff': eff})
 
Enrich_UNAT_ADD = {'name': 'Enrichment_ADD',
           'config': {'Separations': {'leftover_commod': 'c_DU',
                                      'throughput': 1e7,
                                      'feedbuf_size': 1e9,
                                      # can't use feed recipe unless we split
                                      # into 2 sep facilities for each commod
                                      'feed_commods':
                                          {'val': ['c_UNAT_ADD']},
                                      'streams': {
                                          'item': [{
                                              'commod': 'c_UOX_ADD', 
                                              'info': {
                                                  'buf_size': 1e100, 'efficiencies':
                                                  {'item': effs_ADD}}}]}
                                      }
                      }
           }



In [14]:
print(Enrich_UNAT)
print(Enrich_UNAT_ADD)

{'name': 'Enrichment', 'config': {'Separations': {'leftover_commod': 'c_DU', 'throughput': 5000000.0, 'feedbuf_size': 1000000000.0, 'feed_commods': {'val': ['c_UNAT']}, 'streams': {'item': [{'commod': 'c_UOX', 'info': {'buf_size': 250000.0, 'efficiencies': {'item': [{'comp': 'U234', 'eff': 0.8480036104315597}, {'comp': 'U235', 'eff': 0.6812073348954103}, {'comp': 'U238', 'eff': 0.09282132068864048}]}}}]}}}}
{'name': 'Enrichment_ADD', 'config': {'Separations': {'leftover_commod': 'c_DU', 'throughput': 10000000.0, 'feedbuf_size': 1000000000.0, 'feed_commods': {'val': ['c_UNAT_ADD']}, 'streams': {'item': [{'commod': 'c_UOX_ADD', 'info': {'buf_size': 1e+100, 'efficiencies': {'item': [{'comp': 'U232', 'eff': 0.9508808638800464}, {'comp': 'U233', 'eff': 0.9100757654516153}, {'comp': 'U234', 'eff': 0.8294355333854383}, {'comp': 'U235', 'eff': 0.6825164922095507}, {'comp': 'U238', 'eff': 0.09281811152615514}]}}}]}}}}


In [15]:
# DU Store
du_store = {'name': 'StorageDepU',
            'config': {'Storage': {'in_commods': {'val': 'c_DU'},
                                   'in_recipe': 'r_DU',
                                   'out_commods': {'val': 'c_DU_str'},
                                     'residence_time' : 0
                                    }
                       }
           }

dep_u = {'name' : 'r_DU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.0025}, 
                      {'id' : 'U238', 'comp' : 0.9975}]
        }


In [16]:
# LWR Cooling chain

lwr_cool = {
    'name': 'UOXCool', 'config': {
        'Storage': {
            'in_commods': {'val': ['c_S_UOX']},
            'out_commods': {'val': 'c_S_UOX_cool'},
            'residence_time': 81}}}

lwr_cool_ADD = {
    'name': 'UOXCool_ADD', 'config': {
        'Storage': {
            'in_commods': {'val': ['c_S_UOX_ADD']},
            'out_commods': {'val': 'c_S_UOX_ADD_cool'},
            'residence_time': 81}}}

lwr_store = {'name': 'UOXStr',
             'config': {'Storage': {'in_commods': {'val': 'c_S_UOX_cool'},
                                    'out_commods': {'val': 'c_S_UOX_stored'},
                                    'residence_time': 0
                                    }
                        }
             }


sink = {
    'name': 'Waste',
    'config': {
        'Sink': {
            'in_commods': {
                'val': [
                    'c_S_UOX_stored',
                    'StoredDepU']}}}}

In [17]:
# Recipes from 100 ppt U232 additive @ beginning of enrichment
from bam_recipe_100ppt import (uox_no232, isos_no232, ff_no232, ff_232, spentff_no232, 
                           spentff_232
                          )

# Main Input File 

In [18]:
control = {'duration' : sim_duration, 
           'startmonth' : 1, 
           'startyear' : 2022,
           #'dt' : 86400, 
           #'explicit_inventory' : True
#            'tolerance_generic' : 1e-12,
#            'tolerance_resource': 1e-12
          }

## EG01

In [19]:
archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Separations'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'DeployInst'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                       ]
             }

In [20]:
# full additive availability at intro -- skip down for ramp-up sim
full_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, eg1_fc_inst]
              }
full_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : full_region,
                            'facility' : [init_uox_source, non_source, source_UNAT, source_UNAT_ADD, 
                                          Enrich_UNAT, Enrich_UNAT_ADD,
                                          du_store, 
                                          lwr_full, lwr_cool, lwr_cool_ADD, lwr_store, sink
                                         ],
                            'recipe' : [dep_u, uox_no232, isos_no232, nat_u, nat_u_additive, 
                                        r_UOX, r_S_UOX, 
                                        r_UOX_ADD, r_S_UOX_ADD
                                       ]
                           }
           }

In [21]:
#run_sim('01_full-add_flat-pwr', full_sim)
#run_sim('01_full-add_grow-pwr', full_sim)

# EG01 --> 23
## Prototype Definitions

In [22]:
# remember that mixer facilities have fake ratios as of 3/9/22
# only making fast fuel without additive, because the additive doesn't make sense for MOX
# (prototypes exist to have a split, but creating both streams does not work as expected)
from bam_eg23_facilities import (eg23_sink, lwr_sep, 
                             sfr_mix_no232, non_sfr_cool, sfr_sep)

In [23]:
sfr = {'name' : 'SFR', 
       'lifetime' : rxtr_life,
       'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['FF_Non']},
                                'fuel_outcommods' : {'val' : ['SpentFF_Non']},
                                'fuel_inrecipes' :  {'val' : ['FF_no232']},
                                'fuel_outrecipes' : {'val' : ['SpentFF_no232']},
                                'fuel_prefs' : {'val' : [1]},
                                'cycle_time' : 14,
                                'refuel_time' : 0,
                                'assem_size' : 7490,
                                'n_assem_core' : 5,
                                'n_assem_batch' : 1,
                                'power_name' : 'PowerSFR',
                                'power_cap' : 400
                               }
                  }
      }

## Regions and Institutions

In [24]:
eg1_23_fc_prototypes = ['Source_UNAT', 'SourceNonIsos', 'Enrichment', 
                        'Source_UNAT_ADD', 'Enrichment_ADD', 
                        'StorageDepU', 
                        'LWR', 'UOXCool', 'UOXCool_ADD', 'Waste', 'SFR'
                       ]

eg1_23_fc_inst = {'name' : 'FCInstEG01-23', 
                  'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'SourceInitUOX'},
                                                      {'number' : 1, 'prototype' : 'Source_UNAT'},
                                                      {'number' : 1, 'prototype' : 'Source_UNAT_ADD'},
                                                      {'number' : 1, 'prototype' : 'SourceNonIsos'},
                                                      {'number' : 1, 'prototype' : 'Enrichment'},
                                                      {'number' : 1, 'prototype' : 'Enrichment_ADD'},
                                                      {'number' : 1, 'prototype' : 'StorageDepU'},
                                                      {'number' : 1, 'prototype' : 'UOXCool'},
                                                      {'number' : 1, 'prototype' : 'UOXCool_ADD'},
                                                      {'number' : 1, 'prototype' : 'Waste'}]
                                          },
                  'config' : {'ManagerInst' : {'prototypes' : {'val' : eg1_23_fc_prototypes}
                                              }
                             }
                 }

eg23_fc_prototypes = ['UOXSep', 'FFMixNon', 'FFCoolNon', 'FFSep']
eg23_nbuilds = [1 for x in range(0, len(eg23_fc_prototypes))]
eg23_buildtimes = [trans_time for x in range(0, len(eg23_fc_prototypes))]
eg23_deploy = {'name' : 'EG23Deploy', 
               'config' : {'DeployInst' : {'prototypes' :  {'val' : eg23_fc_prototypes},
                                           'n_build' :     {'val' : eg23_nbuilds},
                                           'build_times' : {'val' : eg23_buildtimes}
                                          }
                          }
              }

In [25]:
from math import log
from math import exp

# exponential power demand
## THIS IS NOT FUNCTIONAL, see 3/17 slides ##
year_grow_rate = 0.01
month_grow_rate = year_grow_rate / 12
a_0 = 100000
exp_str_0 = str(a_0) + ' ' + str(month_grow_rate) + ' 0'
exp_func_0 = {'type' : 'exponential', 'params' : exp_str_0}

# exponential transition too:
# in theory the deployment of SFR doesn't need to be exponential but since
# the power commod is linked to this, this is how it will be for now. 
# maybe manual deployments do make the most sense to avoid this constraint
#
# todo: automate getting a_trans from previously calculated power curve
# ...will be especially beneficial for ramp-up approach
pwr_trans = 114000
t_1a = trans_time



a_trans = pwr_trans / np.exp(-month_grow_rate * t_1a)
coeff_decay = log(1-year_grow_rate)/12.


exp_str_1 = str(a_trans) + ' ' + str(coeff_decay) + ' 0'
exp_func_1 = {'type' : 'exponential', 'params' : exp_str_1}

t1b = rxtr_life-old_lives[0]
print(t1b/12.)
a_trans_1b = a_trans * np.exp(-month_grow_rate * (t1b))
exp_str_1b = str(a_trans) + ' -' + str(20*month_grow_rate) + ' 0'
exp_func_1b = {'type' : 'exponential', 'params' : exp_str_1b}

t1c = trans_time+rxtr_life
a_trans_1c = a_trans_1b * np.exp(-10*month_grow_rate * (t1c))
exp_str_1c = str(a_trans) + ' -' + str(1*month_grow_rate) + ' 0'
exp_func_1c = {'type' : 'exponential', 'params' : exp_str_1c}


coeff_decay = log(1-2*year_grow_rate)/12.

exp_str_2 = str(a_trans) + ' ' + str(-coeff_decay) + ' -' + str(a_trans)
exp_func_2 = {'type' : 'exponential', 'params' : exp_str_2}

lin_func_0 = {'type' : 'linear', 'params' : '0 100000'}
exp_func_lwr = {'piece' : [
                           {'start' : 18,
                            'function' : exp_func_0
                           },
                           {'start' : trans_time,
                            'function' : exp_func_1
#                            },
#                            {'start' : t1b,
#                             'function' : exp_func_1b
#                            },
#                            {'start' : t1c,
#                             'function' : exp_func_1c
                            }
                          ]
               }
exp_func_sfr = {'piece' : [{'start' : trans_time,
                            'function' : exp_func_2
                           }
                          ]
               }
growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'PowerLWR',
                                                          'piecewise_function' : exp_func_lwr
                                                         },
                                                         {'commod' : 'PowerSFR',
                                                          'piecewise_function' : exp_func_sfr
                                                         }
                                                        ]
                                              }
                                  }
                }

24.916666666666668


In [26]:
print(exp_str_2)

153883.904064 0.001683558943126622 -153883.904064


In [27]:
archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Separations'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Separations'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'DeployInst'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                      ]
             }





In [28]:
def build_input(unblended_UF6=2e8, no_additive_throuput=5e6, grow_rate = 0.01):
    
    year_grow_rate = grow_rate
    month_grow_rate = year_grow_rate / 12
    a_0 = 100000
    exp_str_0 = str(a_0) + ' ' + str(month_grow_rate) + ' 0'
    exp_func_0 = {'type' : 'exponential', 'params' : exp_str_0}

    # exponential transition too:
    # in theory the deployment of SFR doesn't need to be exponential but since
    # the power commod is linked to this, this is how it will be for now. 
    # maybe manual deployments do make the most sense to avoid this constraint
    #
    # todo: automate getting a_trans from previously calculated power curve
    # ...will be especially beneficial for ramp-up approach
    pwr_trans = 114000
    t_1a = trans_time



    a_trans = pwr_trans / np.exp(-month_grow_rate * t_1a)
    coeff_decay = log(1-year_grow_rate)/12.


    exp_str_1 = str(a_trans) + ' ' + str(coeff_decay) + ' 0'
    exp_func_1 = {'type' : 'exponential', 'params' : exp_str_1}

    t1b = rxtr_life-old_lives[0]
    print(t1b/12.)
    a_trans_1b = a_trans * np.exp(-month_grow_rate * (t1b))
    exp_str_1b = str(a_trans) + ' -' + str(20*month_grow_rate) + ' 0'
    exp_func_1b = {'type' : 'exponential', 'params' : exp_str_1b}

    t1c = trans_time+rxtr_life
    a_trans_1c = a_trans_1b * np.exp(-10*month_grow_rate * (t1c))
    exp_str_1c = str(a_trans) + ' -' + str(1*month_grow_rate) + ' 0'
    exp_func_1c = {'type' : 'exponential', 'params' : exp_str_1c}


    coeff_decay = log(1-2*year_grow_rate)/12.

    exp_str_2 = str(a_trans) + ' ' + str(-coeff_decay) + ' -' + str(a_trans)
    exp_func_2 = {'type' : 'exponential', 'params' : exp_str_2}

    lin_func_0 = {'type' : 'linear', 'params' : '0 100000'}
    exp_func_lwr = {'piece' : [
                               {'start' : 18,
                                'function' : exp_func_0
                               },
                               {'start' : trans_time,
                                'function' : exp_func_1
                                }
                              ]
                   }
    exp_func_sfr = {'piece' : [{'start' : trans_time,
                                'function' : exp_func_2
                               }
                              ]
                   }
    growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'PowerLWR',
                                                              'piecewise_function' : exp_func_lwr
                                                             },
                                                             {'commod' : 'PowerSFR',
                                                              'piecewise_function' : exp_func_sfr
                                                             }
                                                            ]
                                                  }
                                      }
                    }
  
    # full additive availability at intro
    full_region = {'name' : 'GrowthRegion', 
                   'config' : growth_region, 
                   'institution' : [init_fleet, eg1_23_fc_inst, eg23_deploy]
                  }
    
    
    source_UNAT = {'name' : 'Source_UNAT',
                   'config' : {'Source' : {'outcommod' : 'c_UNAT',
                                           'outrecipe' : 'r_UNAT',
                                           'throughput' : 1.8e11,
                                           'inventory_size' : unblended_UF6
                                          }
                              }
                  }

    # ENRICHMENT NO-ADDITIVE
    feed_needed = 10.304 # From URENCO to prod 1kg of UOX at 5% with Tail at 0.25% 7.923 SWU/kg
    effs = []
    for nuc in UNAT_COMP:
        eff = (UOX_COMP[nuc])/(UNAT_COMP[nuc]*feed_needed)
        effs.append({'comp': nuc, 'eff': eff})

    Enrich_UNAT = {'name': 'Enrichment',
               'config': {'Separations': {'leftover_commod': 'c_DU',
                                          'throughput': no_additive_throuput,
                                          'feedbuf_size': no_additive_throuput*12.,
                                          # can't use feed recipe unless we split
                                          # into 2 sep facilities for each commod
                                          'feed_commods':
                                              {'val': ['c_UNAT']},
                                          'streams': {
                                              'item': [{
                                                  'commod': 'c_UOX', 'info': {
                                                      'buf_size': no_additive_throuput, 'efficiencies':
                                                      {'item': effs}}}]}
                                          }
                          }
               }


    
    full_sim = {'simulation' : {'control' : control,
                                'archetypes' : archetypes,
                                'region' : full_region,
                                'facility' : [init_uox_source, non_source, source_UNAT, source_UNAT_ADD, 
                                              Enrich_UNAT, Enrich_UNAT_ADD,
                                              du_store, 
                                              lwr_full, lwr_cool, lwr_cool_ADD, lwr_sep, sfr_mix_no232, sfr,
                                              sfr_sep, non_sfr_cool, eg23_sink
                                             ],
                                'recipe' : [dep_u, uox_no232, isos_no232, nat_u, nat_u_additive, 
                                            r_UOX, r_S_UOX, 
                                            r_UOX_ADD, r_S_UOX_ADD, 
                                            ff_no232, spentff_no232 
                                           ]
                               }
               }
    return full_sim

In [29]:
#run_sim('01_full-add_flat-pwr', full_sim)
from os.path import exists

rerun = False
unblended_list = [2.5e7, 5e7, 7.5e7, 1e8, 1.75e8, 2.5e8, 3.75e8, 5e8]
throughput_list = [2.5e5, 5e5, 7.5e5, 1e6, 1.25e6, 1.75e6, 2.5e6]
growth_list = [0, 0.005, 0.01, 0.015, 0.02]

for i, unblended in enumerate(unblended_list):
    for j, throughput in enumerate(throughput_list):
        for growth in growth_list:
            full_sim = build_input(unblended, throughput, growth)
            output_str = "eg23" 
            output_str += "_grow_" + str(int(unblended)) 
            output_str += "_" + str(int(throughput)) 
            output_str += "_" + str(float(growth*100)) 

            sim_file = '../output/' + output_str + '.sqlite'

            if (not exists(sim_file)) or rerun:
                run_sim(output_str, full_sim)

24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666666668
24.916666666